### Reading Images in a single folder.
This case arises when images of different classes are in the same folder and the classes are detemined by the file name for example if the prefix of the image is `dog` then that image will belong to the class of dog. In this notebook we will go through the mnist images with the following structure.
```
mnist
    - 0_1.png
    - 0_2.png
    - 1_1.png
    ...
```

In this case the first digit of the file name detemines the class where the image belongs to.

In [50]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import pathlib

In [71]:
root_path = "mnist/"

In [80]:
ds_train = tf.data.Dataset.list_files(str(pathlib.Path(root_path, '*.jpg')))
ds_train

<ShuffleDataset shapes: (), types: tf.string>

> Processing the image paths.

In [93]:
def process_path(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=1, dtype="float32")
    label = tf.strings.split(path, "\\")[-1]
    label = tf.strings.substr(label, pos=0, len=1)
    label = tf.strings.to_number(label, out_type=tf.int64)
    return image, label

> Data argumentation on images.

In [94]:
def data_argumentation(image, label):
    image = tf.image.random_brightness(image, 0.05)
    return image, label

In [95]:
ds_train = ds_train.map(process_path).batch(2)
ds_train

<BatchDataset shapes: (<unknown>, (None,)), types: (tf.float32, tf.int64)>

> Trainning the network on our custom image dataset.

In [97]:
model_1 = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation='softmax')
] ,name="model_1")

model_1.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = "adam",
    metrics=["accuracy"]
)
model_1.fit(ds_train, epochs=2, verbose=1)

Epoch 1/2
25/25 [==============================] - 1s 3ms/step - loss: 2.3127 - accuracy: 0.0413
Epoch 2/2
25/25 [==============================] - 0s 3ms/step - loss: 1.9155 - accuracy: 0.6309


That's how we can load images, that has a class name depending on their file naming.